In [1]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow import keras

2022-12-07 23:19:23.927286: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = pd.read_csv("../Data/pharmacy_tx.csv")
df.head()

,tx_date,pharmacy,diagnosis,drug,bin,pcn,group,rejected,patient_pay
0,2022-01-02,Pharmacy #6,G99.93,branded tanoclolol,725700,1UQC,NaN,False,13.39
1,2022-01-02,Pharmacy #42,U60.52,branded oxasoted,664344,NaN,52H8KH0F83K,False,7.02
2,2022-01-02,Pharmacy #37,Q85.91,branded cupitelol,725700,1UQC,NaN,False,13.39
3,2022-01-02,Pharmacy #30,U60.52,generic oxasoted,571569,KB38N,6BYJBW,False,10.84
4,2022-01-02,Pharmacy #18,N55.01,branded mamate,664344,NaN,ZX2QUWR,False,47.00


In [3]:
# Swapping each date in the dataset with the number of days since the first date in the dataset

df['tx_date'] = pd.to_datetime(df['tx_date'])
df['tx_date'] = df['tx_date'] - df['tx_date'].min()
df['tx_date'] = df['tx_date'].dt.days
df['tx_date']

0             0
1             0
2             0
3             0
4             0
           ... 
13910239    362
13910240    362
13910241    362
13910242    362
13910243    362
Name: tx_date, Length: 13910244, dtype: int64

In [5]:
# Drop the pharmacy column

df = df.drop(['pharmacy'], axis=1)
df.head()

,tx_date,diagnosis,drug,bin,pcn,group,rejected,patient_pay
0,0,G99.93,branded tanoclolol,725700,1UQC,NaN,False,13.39
1,0,U60.52,branded oxasoted,664344,NaN,52H8KH0F83K,False,7.02
2,0,Q85.91,branded cupitelol,725700,1UQC,NaN,False,13.39
3,0,U60.52,generic oxasoted,571569,KB38N,6BYJBW,False,10.84
4,0,N55.01,branded mamate,664344,NaN,ZX2QUWR,False,47.00


In [7]:
# Separating drug column into drug name (second word) column and branded or generic (first word)
# column

df['drug_name'] = df['drug'].str.split(' ', 1).str[1]
df['branded_or_generic'] = df['drug'].str.split(' ', 1).str[0]
df.head()

,tx_date,diagnosis,drug,bin,pcn,group,rejected,patient_pay,drug_name,branded_or_generic
0,0,G99.93,branded tanoclolol,725700,1UQC,NaN,False,13.39,tanoclolol,branded
1,0,U60.52,branded oxasoted,664344,NaN,52H8KH0F83K,False,7.02,oxasoted,branded
2,0,Q85.91,branded cupitelol,725700,1UQC,NaN,False,13.39,cupitelol,branded
3,0,U60.52,generic oxasoted,571569,KB38N,6BYJBW,False,10.84,oxasoted,generic
4,0,N55.01,branded mamate,664344,NaN,ZX2QUWR,False,47.00,mamate,branded


In [8]:
df = df.drop(['drug'], axis=1)

In [9]:
df.head()

,tx_date,diagnosis,bin,pcn,group,rejected,patient_pay,drug_name,branded_or_generic
0,0,G99.93,725700,1UQC,NaN,False,13.39,tanoclolol,branded
1,0,U60.52,664344,NaN,52H8KH0F83K,False,7.02,oxasoted,branded
2,0,Q85.91,725700,1UQC,NaN,False,13.39,cupitelol,branded
3,0,U60.52,571569,KB38N,6BYJBW,False,10.84,oxasoted,generic
4,0,N55.01,664344,NaN,ZX2QUWR,False,47.00,mamate,branded


In [10]:
# Only keeping rows with False for rejected column

df = df[df['rejected'] == False]
df.head()

,tx_date,diagnosis,bin,pcn,group,rejected,patient_pay,drug_name,branded_or_generic
0,0,G99.93,725700,1UQC,NaN,False,13.39,tanoclolol,branded
1,0,U60.52,664344,NaN,52H8KH0F83K,False,7.02,oxasoted,branded
2,0,Q85.91,725700,1UQC,NaN,False,13.39,cupitelol,branded
3,0,U60.52,571569,KB38N,6BYJBW,False,10.84,oxasoted,generic
4,0,N55.01,664344,NaN,ZX2QUWR,False,47.00,mamate,branded


In [11]:
df['rejected'].value_counts()

False    12823384
Name: rejected, dtype: int64

In [13]:
df = df.drop(['rejected'], axis=1)

In [15]:
df = df.drop(columns = ['group', 'pcn'])
df['bin '] = df['bin'].map(str)
df.head()

,tx_date,diagnosis,bin,patient_pay,drug_name,branded_or_generic,bin
0,0,G99.93,725700,13.39,tanoclolol,branded,725700
1,0,U60.52,664344,7.02,oxasoted,branded,664344
2,0,Q85.91,725700,13.39,cupitelol,branded,725700
3,0,U60.52,571569,10.84,oxasoted,generic,571569
4,0,N55.01,664344,47.00,mamate,branded,664344


In [17]:
df = df.drop(['bin'], axis=1)
df

,tx_date,diagnosis,patient_pay,drug_name,branded_or_generic,bin
0,0,G99.93,13.39,tanoclolol,branded,725700
1,0,U60.52,7.02,oxasoted,branded,664344
2,0,Q85.91,13.39,cupitelol,branded,725700
3,0,U60.52,10.84,oxasoted,generic,571569
4,0,N55.01,47.00,mamate,branded,664344
...,...,...,...,...,...,...
13910238,362,Q72.66,66.47,momudobatin,branded,571569
13910240,362,N59.44,6.28,tafistitrisin,generic,664344
13910241,362,W50.87,6.94,tanoclolol,generic,691847
13910242,362,I68.27,13.93,prazinib,branded,96934


In [24]:
# Reordering columns to have patient_pay as the last column

df = df[['tx_date', 'diagnosis', 'drug_name', 'branded_or_generic', 'bin ', 'patient_pay']]
df.head()

,tx_date,diagnosis,drug_name,branded_or_generic,bin,patient_pay
0,0,G99.93,tanoclolol,branded,725700,13.39
1,0,U60.52,oxasoted,branded,664344,7.02
2,0,Q85.91,cupitelol,branded,725700,13.39
3,0,U60.52,oxasoted,generic,571569,10.84
4,0,N55.01,mamate,branded,664344,47.00


In [25]:
# One hot encoding the diagnosis, drug_name, branded_or_generic columns, and bin  columns

df = pd.get_dummies(df, columns=['diagnosis', 'drug_name', 'branded_or_generic', 'bin '])
df.head()

,tx_date,patient_pay,diagnosis_A00.82,diagnosis_A13.39,diagnosis_A14.01,diagnosis_A22.87,diagnosis_A45.07,diagnosis_B03.27,diagnosis_B05.36,diagnosis_B42.10,...,bin _539437,bin _571569,bin _664344,bin _691847,bin _718350,bin _725700,bin _756120,bin _757349,bin _956971,bin _96934
0,0,13.39,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,7.02,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0,13.39,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,10.84,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,0,47.00,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [27]:
X = df.drop(['patient_pay'], axis=1)
y = df['patient_pay']

X.head()

,tx_date,diagnosis_A00.82,diagnosis_A13.39,diagnosis_A14.01,diagnosis_A22.87,diagnosis_A45.07,diagnosis_B03.27,diagnosis_B05.36,diagnosis_B42.10,diagnosis_B45.03,...,bin _539437,bin _571569,bin _664344,bin _691847,bin _718350,bin _725700,bin _756120,bin _757349,bin _956971,bin _96934
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [28]:
# Saving X and y as csv files titled "one_hot_encoded_features.csv" and "one_hot_encoded_labels.csv"

X.to_csv('one_hot_encoded_features.csv', index=False)
y.to_csv('one_hot_encoded_labels.csv', index=False)



In [29]:
df = pd.read_csv("../Data/pharmacy_tx.csv")
df.head()

,tx_date,pharmacy,diagnosis,drug,bin,pcn,group,rejected,patient_pay
0,2022-01-02,Pharmacy #6,G99.93,branded tanoclolol,725700,1UQC,NaN,False,13.39
1,2022-01-02,Pharmacy #42,U60.52,branded oxasoted,664344,NaN,52H8KH0F83K,False,7.02
2,2022-01-02,Pharmacy #37,Q85.91,branded cupitelol,725700,1UQC,NaN,False,13.39
3,2022-01-02,Pharmacy #30,U60.52,generic oxasoted,571569,KB38N,6BYJBW,False,10.84
4,2022-01-02,Pharmacy #18,N55.01,branded mamate,664344,NaN,ZX2QUWR,False,47.00


In [30]:
df['tx_date'] = pd.to_datetime(df['tx_date'])
df['tx_date'] = df['tx_date'] - df['tx_date'].min()
df['tx_date'] = df['tx_date'].dt.days
df = df.drop(['pharmacy'], axis=1)
df['drug_name'] = df['drug'].str.split(' ', 1).str[1]
df['branded_or_generic'] = df['drug'].str.split(' ', 1).str[0]
df = df.drop(['drug'], axis=1)
df = df[df['rejected'] == True]
df = df.drop(columns = ['group', 'pcn'])
df['bin '] = df['bin'].map(str)
df = df.drop(['bin'], axis=1)
df = df[['tx_date', 'diagnosis', 'drug_name', 'branded_or_generic', 'bin ', 'patient_pay']]
df.head()

,tx_date,diagnosis,drug_name,branded_or_generic,bin,patient_pay
12,0,M42.24,gentipapavir,branded,725700,0.0
34,0,I38.43,sorine,branded,322463,0.0
43,0,I68.27,mule,branded,160389,0.0
58,0,H60.83,glycogane,branded,664344,0.0
76,0,H36.57,semufolic,branded,664344,0.0


In [31]:
df = pd.get_dummies(df, columns=['diagnosis', 'drug_name', 'branded_or_generic', 'bin '])
df.head()

,tx_date,patient_pay,diagnosis_A13.39,diagnosis_A14.01,diagnosis_A22.87,diagnosis_A45.07,diagnosis_B03.27,diagnosis_B05.36,diagnosis_B42.10,diagnosis_B45.03,...,bin _539437,bin _571569,bin _664344,bin _691847,bin _718350,bin _725700,bin _756120,bin _757349,bin _956971,bin _96934
12,0,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
34,0,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43,0,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
58,0,0.0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
76,0,0.0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [33]:
df.shape

(1086860, 200)

In [34]:
X.shape

(12823384, 225)

In [36]:
X_2 = df.drop(['patient_pay'], axis=1)
y_2 = df['patient_pay']

X_2.head()

12    0.0
34    0.0
43    0.0
58    0.0
76    0.0
Name: patient_pay, dtype: float64

In [37]:
X_2.shape

(1086860, 199)

In [39]:
# Add 26 new columns to X_2 of 0s to match the shape of X using a for loop

for i in range(26):
    X_2[i] = 0


In [41]:
X_2.shape

(1086860, 226)

In [42]:
X_2 = X_2.drop(columns = [0])

In [43]:
X_2.shape

(1086860, 225)

In [44]:
# Save X_2 as a csv file titled "one_hot_encoded_features_rejected.csv"

X_2.to_csv('one_hot_encoded_features_rejected.csv', index=False)
